In [1]:
import os
import sys
from PIL import Image
import numpy as np
import tensorflow.keras as keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [43]:
def loadData(path_to_dataset='Data/leapGestRecog'):
    x = []
    y = []
    if path_to_dataset.endswith('leapGestRecog'):
        for subject in sorted(os.listdir(path_to_dataset)):
            if subject.startswith('.'):
                continue
            print('processing subject {}'.format(subject))
            path_to_gestures = '{}/{}'.format(path_to_dataset, subject)
            for gesture in os.listdir(path_to_gestures):
                if gesture.startswith('.'):
                    continue
                path_to_imgs = '{}/{}'.format(path_to_gestures, gesture)
                for img in os.listdir(path_to_imgs):
                    if img.startswith('.'):
                        continue
                    im = Image.open('{}/{}'.format(path_to_imgs, img)).convert('L').resize((128, 128)) # read, convert to b&w, resize
                    #im = np.stack((np.array(im),)*3, axis=-1)
                    x.append(np.array(im))
                    y.append(gesture[gesture.index('_')+1:])
    elif path_to_dataset.endswith('handGestures'):
        for gesture in os.listdir(path_to_dataset):
            if gesture.startswith('.'):
                continue
            path_to_imgs = '{}/{}'.format(path_to_dataset, gesture)
            for img in os.listdir(path_to_imgs):
                if img.startswith('.'):
                        continue
                im = Image.open('{}/{}'.format(path_to_imgs, img)).resize((128, 128)) # read, convert to b&w, resize
                x.append(np.array(im))
                y.append(gesture)
    else:
        raise Exception('Invalid Dataset!')
        
    return np.array(x), np.array(y)


def preprocessData(x, y):
    data = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
    le = LabelEncoder()
    labels = keras.utils.to_categorical(le.fit_transform(y))
    xTrain, xTest, yTrain, yTest = train_test_split(data, labels)
    return xTrain, xTest, yTrain, yTest, le




In [58]:
dataset = 'handGestures'

In [59]:
x, y = loadData('Data/{}'.format(dataset))
xTrain, xTest, yTrain, yTest, le = preprocessData(x, y)
print('xTrain Shape: {}'.format(xTrain.shape))
print('xTest Shape: {}'.format(xTest.shape))
print('yTrain Shape: {}'.format(yTrain.shape))
print('yTest Shape: {}'.format(yTest.shape))

xTrain Shape: (3932, 128, 128, 1)
xTest Shape: (1311, 128, 128, 1)
yTrain Shape: (3932, 10)
yTest Shape: (1311, 10)


In [60]:
# pickle data to avoid reloading and processing

import pickle

objs = [xTrain, xTest, yTrain, yTest, le]
pickle.dump(objs, open('data_{}.pkl'.format(dataset), 'wb'))